In [1]:
%%capture --no-display
from dask.distributed import Client
client = Client(address = 'tcp://192.168.62.207:8686')
client

Client Scheduler: tcp://192.168.62.207:8686 Dashboard: http://192.168.62.207:8687/status,Cluster Workers: 14 Cores: 14 Memory: 942.82 GB


In [32]:
import takco
fnames = '/export/scratch1/home/kruit/scratch/output/chartedIn/9-triples/*.jsonl'
tables = takco.DaskHashBag(client=client).load(fnames).persist()
tables.bag.count().compute()

35

In [33]:
biggest = sorted(tables, key=lambda t: -len(t.body))
takco.preview(biggest)

?,0,1,2,3
∈,Q2031291 Q134556,decimal,dateTime,Thing
0,,,,P2291
3,P527,,,
,_pgTitle,Peak position,_Variable,Chart
,Somebody That I Used to Know,21,2012,Canada ( Canadian Hot 100 )
,Somebody That I Used to Know,48,2012,UK Singles ( OCC )
,Somebody That I Used to Know,26,2012,US Billboard Hot 100
,Oops!... I Did It Again (song),1,2000,Australia ( ARIA )
,Oops!... I Did It Again (song),2,2000,Austria ( Ö3 Austria Top 40 )
?,0,1,2,3
